# Libraries

In [1]:
import numpy as np
import pandas as pd

# Data

In [6]:
data = pd.read_csv("./data/wina.pl_data.csv")
data.head()

,name,link,price,kraj,region,apelacja,winnica,rodzaj wina,smak,styl,...,rocznik3,temperatura podawania2,wino koszerne,region2,ilość win,grona9,grona10,grona11,grona12,grona13
0,"1,5 L Szampan Moet & Chandon Brut Imperial Mag...",https://www.wina.pl/szampany/3056-15-l-szampan...,"666,00 zł",Francja,Szampania,Champagne AOC,Moet & Chandon,szampan,wytrawne,średnie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11 Filari Primitivo di Manduria San Marzano NV...,https://www.wina.pl/slodkie/2869-11-filari-pri...,"109,00 zł",Włochy,Apulia,Primitivo di Manduria DOP,NaN,czerwone,słodkie,pełne,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3 L Szampan Moet & Chandon Brut Imperial Jerob...,https://www.wina.pl/szampany/4154-3-l-szampan-...,"1 999,00 zł",Francja,Szampania,Champagne AOC,Moet & Chandon,szampan,wytrawne,średnie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,770 Miles Zinfandel California 2021,https://www.wina.pl/wina-czerwone-wytrawne/451...,"37,00 zł",USA,Kalifornia,NaN,NaN,czerwone,wytrawne,średnie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,770 Miles Zinfandel Rose California 2021,https://www.wina.pl/wina-polwytrawne/4528-770-...,"39,00 zł",USA,Kalifornia,NaN,NaN,różowe,półwytrawne,lekkie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2251 entries, 0 to 2250
Data columns (total 52 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    2251 non-null   object 
 1   link                    2251 non-null   object 
 2   price                   2251 non-null   object 
 3   kraj                    2242 non-null   object 
 4   region                  2079 non-null   object 
 5   apelacja                1650 non-null   object 
 6   winnica                 1231 non-null   object 
 7   rodzaj wina             2216 non-null   object 
 8   smak                    2197 non-null   object 
 9   styl                    1808 non-null   object 
 10  grona                   2197 non-null   object 
 11  pojemność               2178 non-null   object 
 12  temperatura podawania   1779 non-null   object 
 13  wysyłka w ciągu         2127 non-null   object 
 14  rocznik                 2030 non-null   